# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

import re
import pickle

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

import warnings

warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterMessages.db')
df = pd.read_sql_table('Messages', engine)
df.head()

X = df.message.values
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """
    Strips the characters that aren't alphanumeric and then processes the tokens

    Args:
    text: string containing message

    Returns:
    stemmed and cleaned tokens
    """
    # strip non-alphanumeric chars and lowercase the text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize to words
    tokens = word_tokenize(text)
    
    # lammatize the tokens
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")

#     clean_tokens = []
#     for tok in tokens:
#         clean_tok = lemmatizer.lemmatize(tok).lower().strip()
#         clean_tokens.append(clean_tok)
        
    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 42)

np.random.seed(33)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def get_eval_metrics(actual, predicted, col_names):
    """Calculate evaluation metrics for ML model
    
    Args:
    actual: array. Array containing actual labels.
    predicted: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the predicted fields.
       
    Returns:
    metrics_df: dataframe. Dataframe containing the accuracy, precision, recall 
    and f1 score for a given set of actual and predicted labels.
    """
    metrics = []
    
    # Calculate evaluation metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(actual[:, i], predicted[:, i])
        precision = precision_score(actual[:, i], predicted[:, i], average='macro')
        recall = recall_score(actual[:, i], predicted[:, i], average='macro')
        f1 = f1_score(actual[:, i], predicted[:, i], average='macro')
        
        metrics.append([accuracy, precision, recall, f1])
    
    # Create dataframe containing metrics
    metrics = np.array(metrics)
    metrics_df = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return metrics_df

In [7]:
Y_train_pred = pipeline.predict(X_train)
col_names = list(Y.columns.values)

print(get_eval_metrics(np.array(Y_train), Y_train_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.990845   0.985462  0.969525  0.977334
request                 0.987133   0.990795  0.963801  0.976676
offer                   0.998627   0.999311  0.841176  0.905250
aid_related             0.983979   0.985507  0.981660  0.983457
medical_help            0.987946   0.992835  0.924089  0.955389
medical_products        0.991456   0.995546  0.913313  0.950306
search_and_rescue       0.994660   0.997266  0.907080  0.947410
security                0.994965   0.997449  0.860563  0.917707
military                0.995830   0.997016  0.938610  0.965859
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.994863   0.997272  0.959729  0.977652
food                    0.994202   0.996547  0.974030  0.984944
shelter                 0.992422   0.995304  0.957532  0.975498
clothing                0.998423   0.999200  0.949511  0.973013
money                   0.995931   0.997

In [8]:
Y_test_pred = pipeline.predict(X_test)

eval_metrics0 = get_eval_metrics(np.array(Y_test), Y_test_pred, col_names)
print(eval_metrics0)

                        Accuracy  Precision    Recall        F1
related                 0.799359   0.600069  0.550573  0.569991
request                 0.885261   0.841399  0.709205  0.750346
offer                   0.994965   0.497482  0.500000  0.498738
aid_related             0.754196   0.750301  0.732381  0.737325
medical_help            0.920201   0.744009  0.546978  0.564620
medical_products        0.949649   0.828401  0.534079  0.550531
search_and_rescue       0.975587   0.723744  0.524454  0.539268
security                0.981996   0.616221  0.504077  0.503791
military                0.969637   0.742356  0.550926  0.579404
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.945377   0.876807  0.592949  0.638885
food                    0.918676   0.886469  0.668610  0.723367
shelter                 0.934544   0.875389  0.666545  0.722537
clothing                0.987183   0.902967  0.591527  0.646766
money                   0.979707   0.740

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
# Define performance metric for use in grid search scoring object
def performance_metric(y_true, y_pred):
    """Calculate median F1 score for all of the output classifiers
    
    Args:
    y_true: array. Array containing actual labels.
    y_pred: array. Array containing predicted labels.
        
    Returns:
    score: float. Median F1 score for all of the output classifiers
    """
    f1_list = []
    for i in range(np.shape(y_pred)[1]):
        f1 = f1_score(np.array(y_true)[:, i], y_pred[:, i], average='macro')
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

In [10]:
# Create grid search object

parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

scorer = make_scorer(performance_metric)
cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer, verbose = 10)

# Find best parameters
np.random.seed(81)
tuned_model = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.5435348882803395, total=  53.4s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.552873531107866, total=  53.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.3min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.5675979886936737, total=  53.9s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  8.5min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1, score=0.5552138251596097, total=  55.4s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  9.7min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.5742938879906486, total=  41.5s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.5663271670132417, total=  41.2s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.5907377937567246, total=  41.5s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1, score=0.5604777984807566, total= 1.9min
[CV] clf__estimator__min_samples_split=2, clf__

[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 95.6min finished


In [11]:

# Get results of grid search
tuned_model.cv_results_

{'mean_fit_time': array([  46.44882234,   34.85331853,   47.56287551,   34.64383698,
         102.21323061,   72.0846231 ,  102.98815521,   71.10837523,
          38.91847706,   31.77689393,   38.82274715,   30.82738908,
          81.96438734,   63.86672791,   80.7208724 ,   61.17300733,
          35.0429608 ,   29.839516  ,   34.86358802,   29.05485765,
          71.92614102,   59.49111438,   71.54929868,   57.1304423 ]),
 'std_fit_time': array([ 0.2120114 ,  0.07386094,  0.4632251 ,  0.12783927,  0.33434777,
         0.22132051,  0.9003926 ,  0.13499275,  0.183641  ,  0.13943674,
         0.01541135,  0.03904305,  0.13858673,  0.30396897,  1.17186422,
         0.20420184,  0.09822844,  0.01953783,  0.25627104,  0.26128957,
         0.33766267,  0.36409989,  0.16324261,  0.09638975]),
 'mean_score_time': array([ 7.25142256,  6.80136704,  7.24962942,  6.75237187,  9.90628815,
         8.74117668,  9.76895118,  8.59326061,  7.22366238,  6.79770343,
         7.18712862,  6.74429297,  9.8

In [12]:
# Parameters for best mean test score
tuned_model.best_params_

{'clf__estimator__min_samples_split': 10,
 'clf__estimator__n_estimators': 10,
 'tfidf__use_idf': True,
 'vect__min_df': 5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
# Calculate evaluation metrics for test set
tuned_pred_test = tuned_model.predict(X_test)

eval_metrics1 = get_eval_metrics(np.array(Y_test), tuned_pred_test, col_names)

print(eval_metrics1)

                        Accuracy  Precision    Recall        F1
related                 0.799969   0.571464  0.545884  0.551565
request                 0.885413   0.834017  0.717177  0.755781
offer                   0.994965   0.497482  0.500000  0.498738
aid_related             0.761062   0.752578  0.752280  0.752427
medical_help            0.922185   0.753445  0.582970  0.615163
medical_products        0.955600   0.884627  0.599002  0.649363
search_and_rescue       0.977266   0.805430  0.574377  0.616057
security                0.981080   0.541213  0.503611  0.503161
military                0.970095   0.752836  0.582639  0.621158
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.961703   0.896094  0.749922  0.803980
food                    0.941562   0.881732  0.808481  0.839863
shelter                 0.932103   0.845850  0.669090  0.720390
clothing                0.987946   0.897630  0.627086  0.690755
money                   0.980012   0.776

In [14]:
# Get summary stats for first model
eval_metrics0.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.944173,0.757346,0.588627,0.607653
std,0.057562,0.152386,0.115889,0.124496
min,0.754196,0.468626,0.499674,0.483652
25%,0.933895,0.667025,0.509762,0.507788
50%,0.958422,0.787919,0.544341,0.564381
75%,0.983293,0.875743,0.633680,0.668397
max,1.000000,1.000000,1.000000,1.000000


In [15]:
# Get summary stats for tuned model
eval_metrics1.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.947030,0.756476,0.619966,0.642752
std,0.055970,0.152866,0.127701,0.135216
min,0.761062,0.493821,0.500000,0.491853
25%,0.940227,0.656962,0.520355,0.528355
50%,0.962618,0.786965,0.580071,0.617378
75%,0.983064,0.863566,0.681112,0.728399
max,1.000000,1.000000,1.000000,1.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
# Pickle best model
pickle.dump(tuned_model, open('DisasterMessagesModel.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.